### List of Hyperlinks

Let's create the local list of files to be parsed.

In [5]:
name = 'pvp'
#name = 'guber'
root = '/home/www/html/immobiliarista/'

In [ ]:
if usecase == 'guber':
    s.replace('="/', '="https://immobiliare.guber.it/')
    #https://immobiliare.guber.it/Immobile/125200 -> http://localhost:8001/guber/guber_125200.html

In [7]:
filenames = {}

In [8]:
pattern = f'/home/www/html/immobiliarista/{name}/{name}_*.html'
filenames['All'] = glob(pattern)
print(len(filenames['All']))

11567


In [9]:
pattern = f'/home/www/html/immobiliarista/{name}/{name}_*_2023*.html'
filenames['Searches'] = glob(pattern)
print(len(filenames['Searches']))

1576


In [10]:
pattern = f'/home/www/html/immobiliarista/{name}/{name}_*_20230402164743_*.html'
filenames['Last Search'] = glob(pattern)
print(len(filenames['Last Search']))

201


In [11]:
pattern = f'/home/www/html/immobiliarista/{name}/{name}_LTT*.html'
filenames['Property'] = glob(pattern)
print(len(filenames['Property']))

9991


Check the number of downloaded files searches and properties is as "expected".

In [12]:
assert(len(filenames['All']) == len(filenames['Searches']) +  len(filenames['Property']))

In [20]:
os.path.relpath(filenames['Last Search'][0], root)

'pvp/pvp_genova_20230402164743_02.html'

In [21]:
filename = '/home/git/property_scraper/demos/pvp_search_default.json'
with open(filename, 'r') as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

{
    "follow_links": true,
    "download_pages": true,
    "root": "https://pvp.giustizia.it/pvp/it/risultati_ricerca.page?",
    "searches": [
        "localita=Bologna&tipo_bene=immobili&categoria=&geo=geografica&nazione=ITA&ordinamento=data_vendita_decre&ordine_localita=a_z&view=list&elementiPerPagina=50",
        "localita=Genova&tipo_bene=immobili&categoria=&geo=geografica&nazione=ITA&ordinamento=data_vendita_decre&ordine_localita=a_z&view=list&elementiPerPagina=50",
        "localita=Palermo&tipo_bene=immobili&categoria=&geo=geografica&nazione=ITA&ordinamento=data_vendita_decre&ordine_localita=a_z&view=list&elementiPerPagina=50",
        "localita=Ravenna&tipo_bene=immobili&categoria=&geo=geografica&nazione=ITA&ordinamento=data_vendita_decre&ordine_localita=a_z&view=list&elementiPerPagina=50"
    ],
    "other searches": [
        "tipo_bene=immobili&categoria=immobile_residenziale&geo=raggio&lat=&lng=&indirizzo=&raggio=25&prezzo_da=&prezzo_a=&tribunale=&procedura=593&anno=2021&

In [22]:
data['follow_links'] = False
data['download_pages'] = False
data['root'] = 'http://localhost:8000/'
data['searches'] = [os.path.relpath(x, root) for x in filenames['Last Search']]

In [23]:
ofilename = '/home/git/property_scraper/demos/pvp_search_localhost.json'
with open(ofilename, 'w') as f:
    json.dump(data, f, indent=4)

Let's enrich the search table with search details.

## Searches

In [29]:
filename = '../demos/pvp_search_default.csv'
rootname, sheetname = os.path.splitext(os.path.basename(filename))[0].split('_')[-2:]
ofilename = f'/home/git/property_scraper/demos/{name}.xlsx'

In [30]:
searches = pd.read_csv(filename)
print(searches.shape)
display(searches.head(3).T)

(201, 14)


,0,1,2
basename,pvp_ravenna_20230402164743_00.html,pvp_genova_20230402164743_00.html,pvp_ravenna_20230402164743_01.html
filename,/home/git/property_scraper/www/pvp/pvp_ravenna...,/home/git/property_scraper/www/pvp/pvp_genova_...,/home/git/property_scraper/www/pvp/pvp_ravenna...
id,pvp_ravenna_20230402164743_00,pvp_genova_20230402164743_00,pvp_ravenna_20230402164743_01
is_downloaded,False,False,False
is_relative_href_fixed,False,False,False
location,ravenna,genova,ravenna
number_of_pages,30,62,30
number_of_results,1459,3069,1459
number_of_results_per_page,50,50,50
page_id,1,1,2


In [34]:

indices = sorted(searches[searches['Numero di Pagine'] == 'number_of_pages'].index)
searches = searches.drop(indices)
#searches['data_di_vendita'] = pd.to_datetime(searches['data_di_vendita'])
for cid in ['Scaricato?', 'Hyperlink relativo riparato?']:
    searches[cid] = searches[cid].astype(bool)
for cid in ['Numero di Pagine', 'Numero di Risultati', 'Numero di Risultati per Pagina', 'Identificativo della Pagina', 'Codice dello Stato della Risposta']:
    searches[cid] = searches[cid].astype(int)
searches = searches.drop_duplicates(keep='first')
print(searches.shape)
display(searches.head(3).T)

(201, 14)


,0,1,2
Basename,pvp_ravenna_20230402164743_00.html,pvp_genova_20230402164743_00.html,pvp_ravenna_20230402164743_01.html
Filename,/home/git/property_scraper/www/pvp/pvp_ravenna...,/home/git/property_scraper/www/pvp/pvp_genova_...,/home/git/property_scraper/www/pvp/pvp_ravenna...
Identificativo,pvp_ravenna_20230402164743_00,pvp_genova_20230402164743_00,pvp_ravenna_20230402164743_01
Scaricato?,False,False,False
Hyperlink relativo riparato?,False,False,False
location,ravenna,genova,ravenna
Numero di Pagine,30,62,30
Numero di Risultati,1459,3069,1459
Numero di Risultati per Pagina,50,50,50
Identificativo della Pagina,1,1,2


In [35]:
searches.dtypes

Basename                             object
Filename                             object
Identificativo                       object
Scaricato?                             bool
Hyperlink relativo riparato?           bool
location                             object
Numero di Pagine                      int64
Numero di Risultati                   int64
Numero di Risultati per Pagina        int64
Identificativo della Pagina           int64
Codice dello Stato della Risposta     int64
Nome del Ragno                       object
URL                                  object
Localhost URL                        object
dtype: object

{'localita': ['Palermo'], 'geo': ['geografica'], 'ordinamento': ['data_vendita_decre'], 'view': ['list'], 'frame4_item': ['63'], 'nazione': ['ITA'], 'elementiPerPagina': ['50'], 'ordine_localita': ['a_z'], 'tipo_bene': ['immobili']}


(201, 24)


,0,1,2
Basename,pvp_ravenna_20230402164743_00.html,pvp_genova_20230402164743_00.html,pvp_ravenna_20230402164743_01.html
Filename,/home/git/property_scraper/www/pvp/pvp_ravenna...,/home/git/property_scraper/www/pvp/pvp_genova_...,/home/git/property_scraper/www/pvp/pvp_ravenna...
Identificativo,pvp_ravenna_20230402164743_00,pvp_genova_20230402164743_00,pvp_ravenna_20230402164743_01
Scaricato?,False,False,False
Hyperlink relativo riparato?,False,False,False
location,ravenna,genova,ravenna
Numero di Pagine,30,62,30
Numero di Risultati,1459,3069,1459
Numero di Risultati per Pagina,50,50,50
Identificativo della Pagina,1,1,2


In [41]:
print(ofilename, sheetname)

with pd.ExcelWriter(ofilename) as f:
    searches.to_excel(f, sheet_name=sheetname, index=False)

/home/git/property_scraper/demos/search.xlsx default


## Search Properties

In [56]:
filename = '/home/git/property_scraper/demos/pvp_search_localhost.json'
with open(filename, 'r') as f:
    data = json.load(f)
#print(json.dumps(data, indent=4))

In [57]:
data['follow_links'] = True
data['download_pages'] = True
data['root'] = 'http://localhost:8000/'
data['searches'] = [os.path.relpath(x, root) for x in filenames['Last Search']]

In [58]:
ofilename = '/home/git/property_scraper/demos/pvp_search_property_localhost.json'
with open(ofilename, 'w') as f:
    json.dump(data, f, indent=4)

## Search Properties

In [7]:
filename = '../demos/pvp_search_property_localhost.csv'

In [11]:
properties = pd.read_csv(filename)
print(properties.shape)
display(properties.head(3).T)

(9930, 19)


,0,1,2
basename,pvp_LTT7949192.html,pvp_LTT7917024.html,pvp_LTT5820958.html
data_della_ricerca,NaN,NaN,NaN
data_di_vendita,11/05/2023 15:00,11/05/2023 15:00,29/10/2021 10:00
filename,/home/git/property_scraper/www/pvp/pvp_LTT7949...,/home/git/property_scraper/www/pvp/pvp_LTT7917...,/home/git/property_scraper/www/pvp/pvp_LTT5820...
id,pvp_LTT7949192,pvp_LTT7917024,pvp_LTT5820958
indirizzo,Via Baracchino 3-3 - 16100 Genova (GE),Via Pieve di Cadore civico 18 interno 17 Genov...,"via guelfa, n. 5e-d Bologna (BO)"
is_downloaded,False,False,False
is_relative_href_fixed,False,False,False
lotto,Lotto nr. 1,LOTTO UNICO,Lotto nr. 13424
modalita_consegna,Sincrona mista,Sincrona mista,Presso il venditore


In [14]:
properties = properties.rename(columns=scraper.SEARCH_NAMES)

indices = sorted(properties[properties['Data di vendita'] == 'Data di vendita'].index)
properties = properties.drop(indices)
properties['Data di vendita'] = pd.to_datetime(properties['Data di vendita'])
for cid in ['Offerta minima', 'Prezzo base', 'Rialzo minimo']:
    properties[cid] = properties[cid].astype(float)

In [15]:
properties.dtypes

Basename                                     object
Data della ricerca                          float64
Data di vendita                      datetime64[ns]
Filename                                     object
Identificativo                               object
Indirizzo                                    object
Scaricato?                                     bool
Hyperlink relativo riparato?                   bool
Lotto                                        object
Modalità consegna                            object
Numero della procedura                       object
Offerta minima                              float64
Prezzo base                                 float64
Codice dello stato della risposta             int64
Rialzo minimo                               float64
Identificativo della ricerca                 object
Nome del ragno                               object
URL                                          object
Localhost URL                                object
dtype: objec

In [17]:
properties = properties.drop_duplicates(keep='first')
print(properties.shape)
display(properties.head(3).T)

(9930, 19)


,0,1,2
Basename,pvp_LTT7949192.html,pvp_LTT7917024.html,pvp_LTT5820958.html
Data della ricerca,NaN,NaN,NaN
Data di vendita,2023-11-05 15:00:00,2023-11-05 15:00:00,2021-10-29 10:00:00
Filename,/home/git/property_scraper/www/pvp/pvp_LTT7949...,/home/git/property_scraper/www/pvp/pvp_LTT7917...,/home/git/property_scraper/www/pvp/pvp_LTT5820...
Identificativo,pvp_LTT7949192,pvp_LTT7917024,pvp_LTT5820958
Indirizzo,Via Baracchino 3-3 - 16100 Genova (GE),Via Pieve di Cadore civico 18 interno 17 Genov...,"via guelfa, n. 5e-d Bologna (BO)"
Scaricato?,False,False,False
Hyperlink relativo riparato?,False,False,False
Lotto,Lotto nr. 1,LOTTO UNICO,Lotto nr. 13424
Modalità consegna,Sincrona mista,Sincrona mista,Presso il venditore


In [20]:
#properties['Data_di_vendita'] = pd.to_datetime(properties['data_di_vendita'])
properties = properties.sort_values(by=['Data di vendita'], ascending=False)
#print(properties.shape)
display(properties.head(3).T)

,6093,6254,6300
Basename,pvp_LTT7930834.html,pvp_LTT7911648.html,pvp_LTT7839819.html
Data della ricerca,NaN,NaN,NaN
Data di vendita,2023-11-07 10:30:00,2023-11-07 10:30:00,2023-11-07 10:30:00
Filename,/home/git/property_scraper/www/pvp/pvp_LTT7930...,/home/git/property_scraper/www/pvp/pvp_LTT7911...,/home/git/property_scraper/www/pvp/pvp_LTT7839...
Identificativo,pvp_LTT7930834,pvp_LTT7911648,pvp_LTT7839819
Indirizzo,"via Dante Benazzi, 1 Bologna (BO)",via Casteldebole 6/2 Bologna (BO),Contrada Macchialonga Bolognano (PE)
Scaricato?,False,False,False
Hyperlink relativo riparato?,False,False,False
Lotto,Lotto nr. 8998,LOTTO UNICO,Lotto nr. 6
Modalità consegna,Presso il venditore,Presso il venditore,Presso il venditore


In [22]:
sheetname = 'search_property'
ofilename = f'/home/git/property_scraper/demos/{name}.xlsx'

print(ofilename, sheetname)

with pd.ExcelWriter(ofilename, engine='openpyxl', mode='a') as f:
    properties.to_excel(f, sheet_name=sheetname, index=False)

#ofilename = filename.replace('.csv', '.xlsx')
#properties.to_excel(ofilename)

/home/git/property_scraper/demos/pvp.xlsx search_property


## Property

In [13]:
filename = '/home/git/property_scraper/demos/pvp_search_property_localhost.json'
with open(filename, 'r') as f:
    data = json.load(f)
#print(json.dumps(data, indent=4))

In [14]:
data['follow_links'] = False
data['download_pages'] = False
data['root'] = 'http://localhost:8000/'
data['searches'] = [os.path.relpath(x, root) for x in filenames['Property']]

In [16]:
ofilename = '/home/git/property_scraper/demos/pvp_property_localhost.json'
with open(ofilename, 'w') as f:
    json.dump(data, f, indent=4)

In [1]:
filename = '/home/git/property_scraper/demos/pvp_property_localhost.csv'

In [5]:
property = pd.read_csv(filename)
display(property.head(3).T)

,0,1,2
allegati,"AVVISO DI VENDITA PUBBLICITA' .pdf (247 Kb),PR...","Provvedimento di delega.pdf (391 Kb),Avviso di...",AVVISO VENDITE FALL 75 GHERO lotti 1 a 11.pdf ...
allegati_url,https://pvp.giustizia.it/pvp-resources/cms/doc...,https://pvp.giustizia.it/pvp-resources/cms/doc...,https://pvp.giustizia.it/pvp-resources/cms/doc...
basename,NaN,NaN,NaN
bene_descrizione,Appartamento sito al 4° piano composto da ingr...,"Piena proprietà dell’appartamento in Palermo, ...",Lotto 11 - Posto auto esterno ubicato all’inte...
bene_disponibilita,NaN,NaN,NaN
bene_foglio,NaN,NaN,NaN
bene_particella,NaN,NaN,NaN
bene_piano,NaN,NaN,NaN
bene_sub,NaN,NaN,NaN
bene_tipologia,Senza incanto,Senza incanto,Senza incanto


In [ ]:
filename = '/home/git/property_scraper/www/guber/guber_20230325190357_001.html'
#guber_20230323200207_001.html'
filenames = [filename]

In [ ]:
#="/
name = 'guber'

="/ -> ="https://immobiliare.guber.it/

="https://immobiliare.guber.it/Immobile/110794 -> ="./guber_110794.html

In [ ]:
func = {'https://pvp.giustizia.it/pvp/it/dettaglio_annuncio.page?': get_filename_from_identifier}
keys = ['contentId']
root = 'http://localhost:8000/{name}/{name}'

In [ ]:
pbar = tqdm(filenames, position=0)
for filename in pbar:
    fix_hyperlinks(filename, func, keys, root, True)

In [ ]:
!ls ../data/*.txt

In [ ]:
filename = '../data/spiders.txt'

In [ ]:
with open(filename, 'r') as f:
    spiders = sorted([x.strip() for x in f.readlines() if not x.startswith('#')])
print('The following scrapy spiders are available:\n\t' + '\n\t'.join(spiders))

In [ ]:
script_filename = '../demos/run_spiders.bash'
with open(script_filename, 'w') as f:
    f.write('#!/bin/bash\n\n')
    for spider in spiders:
        folder = f"../www/{spider}"
        if not os.path.exists(folder):
            os.makedirs(folder)

        command = f"echo scrapy runspider ../property_scraper/{spider}/spiders/{spider}.py --output ../demos/{spider}.csv"
        f.write(command + '\n')
        #run(command, shell=True, check=True) #, bufsize=bufsize, stdin=PIPE, stdout=PIPE, close_fds=True)
    f.write('')

In [ ]:
stop here

In [ ]:
url = 'https://www.atico.es/resultados-de-la-busqueda/page/2/'


In [ ]:
search_filename = '../demos/alltownri.csv'
properties = pd.read_csv(search_filename)
display(properties.head(3).T)

In [ ]:
name = 'immobiliare'

In [ ]:
s = 'https://www.idealista.it/vendita-case/ravenna-ravenna/'
s.split('/')[4]

In [ ]:
s = 'https://www.immobiliare.it/vendita-case/ravenna-provincia/?criterio=rilevanza'
s.split('/')[4]

### PVP

In [ ]:
#filenames = glob('/home/git/property_scraper/www/pvp/pvp_LTT*.html')
#_filenames = glob('/home/git/property_scraper/www/pvp/pvp_*.html')
#filenames = [x for x in _filenames if 'pvp_LTT' not in x]
filenames = glob(f'/home/git/property_scraper/www/{name}/{name}_*.html')
print(len(filenames))

In [ ]:
func = {'https://pvp.giustizia.it/pvp/it/dettaglio_annuncio.page?': get_filename_from_identifier}
keys = ['contentId']
root = 'http://localhost:8000/{name}/{name}'

In [ ]:
pbar = tqdm(list(reversed(filenames)), position=0)
for filename in pbar:
    fix_hyperlinks(filename, func, keys, root, True)

### Intrum

In [ ]:
name = 'intrum'
#filenames = glob('/home/git/property_scraper/www/pvp/pvp_LTT*.html')
#_filenames = glob('/home/git/property_scraper/www/pvp/pvp_*.html')
#filenames = [x for x in _filenames if 'pvp_LTT' not in x]
filenames = glob(f'/home/git/property_scraper/www/{name}/{name}_*.html')
print(len(filenames))

In [ ]:
func = {}
keys = []
root = 'http://localhost:8000/{name}/{name}'

In [ ]:

pbar = tqdm(filenames, position=0)

for filename in pbar:
    with open(filename, 'r') as f:
        content = f.read()
        soup = convert_hyperlink_rel2global(content, rootname)
    ofilename = filename
    with open(ofilename, 'wb') as f:
        f.write(soup.prettify("utf-8"))        